[MIT License](https://opensource.org/licenses/MIT)

Copyright 2018 Matt Jadud (matt@jadud.com)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import plotly.plotly as py
import pandas as pd
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
from geopy import distance
# distance.distance(tup, tup) takes a pair of (lat, lon) and 
# calculates the distance between them in km.
from collections import namedtuple as namedtuple
from time import sleep
from IPython.core.display import display, HTML

ModuleNotFoundError: No module named 'plotly'

In [2]:
geo          = namedtuple('geo', ['name', 'lat', 'lon'])
marker_style = namedtuple('marker_style', ['size', 'color', 'opacity'])

In [3]:
class EasyMap ():
    # This is the DCS 103 public token. It can be changed if needed.
    # That is, if it gets out in the wild, then we'll want to swap it/remove it.
    token = ("nopenopenope")
    meta = {}
    # Our default style is a reasonable sized dot in Bates Garnet.
    marker_style = marker_style(8, '#881124', 1)
    
    def __init__(self, map_name, username):
        self.meta["map_name"] = map_name
        self.meta["username"] = username
        self.set_defaults()
        
    # Sets the MapBox API token.
    def set_token(self, tok):
        self.token = tok
    
    def set_geocode_delay(secs):
        self.meta["geocode_delay"] = secs
        
    def set_defaults(self):
        # Must be a MapBox style URL.
        self.meta["style"] = "mapbox://styles/mjadud/cjmyx30qr32g02rpcsrunes4v"
        self.meta["locator"] = Nominatim(user_agent="PPPP " + self.meta["username"])
        # Create an empty dictionary of geopoints.
        self.meta["geos"] = dict()
        # Center this on Lewistion, Maine
        # We'll do this the hard way, because I don't want it plotted on the map.
        self.meta["center"] = geo("Lewiston, ME", 44.100351, -70.2147764)
        # Make sure we're ready to accommodate marker styles.
        self.meta["category_styles"] = dict()
        self.meta["category_styles"]["default"] = self.marker_style
        self.meta["geocode_delay"] = 0.5
    
    # CONTRACT
    # set_center : string -> geo
    # PURPOSE
    # Lets us recenter the map. Does not add it to the list of points
    # that we are plotting. Returns the geo of the point.
    def set_center(self, addr):
        g = None
        pt = self.meta["locator"].geocode(addr)
        if not pt:
            print("Could not look up the address [{0}]".format(addr))
            return None
        else:
            g  = geo(addr, pt.latitude, pt.longitude)
            self.meta["center"] = g
        return g
        
    # CONTRACT
    # set_style : url -> _
    # PURPOSE
    # Consumes a MapBox style URL, and sets it for configuring 
    # the output map via Plotly.
    def set_style(self, style_url):
        self.meta.style = style_url
    
    # CONTRACT
    # set_category_style : mstyle -> _
    # PURPOSE
    # Takes an mstyle, and stores it for marker plotting.
    def set_category_style(self, cat, size, color, opacity):
        self.meta["category_styles"][cat] = marker_style(size, color, opacity)
        
    # CONTRACT
    # add_point : string number number -> geo
    # PURPOSE
    # Adds a point to the map. Returns a geo structure of the lat/lon given.
    def add_point(self, name, lat, lon, cat = "default"):
        
        if cat in self.meta["geos"]:
            self.meta["geos"][cat][name] = geo(name, lat, lon)
        else:
            self.meta["geos"][cat] = dict()
            self.meta["geos"][cat][name] = geo(name, lat, lon)
            
        return self.meta["geos"][cat][name]
    
    # CONTRACT
    # add_address : string -> geo
    # PURPOSE
    # Adds a point to the map based on an address.
    # Returns the geo associated with that point.
    def add_address(self, addr, cat = "default"):
        # geolocator.geocode("2 Andrews Road Lewiston Maine")
        pt = self.meta["locator"].geocode(addr)
        sleep(self.meta["geocode_delay"])
        if not pt:
            print("Could not look up the address [{0}]".format(addr))
            return None
        else:
            return self.add_point(addr, pt.latitude, pt.longitude, cat = cat)
        
    
    def setup_map(self):
        self.meta["data"] = []
        data = []
        
        # To build up the data for the plot, we need each category 
        # of points to be a different style. This requires walking each
        # category separately.
        
        # The key is the category, the d is a dict of geos.
        for cat, d in self.meta["geos"].items():
            lats  = list()
            lons  = list()
            texts = list()
            for k, v in d.items():
                texts.append(k)
                lats.append(v.lat)
                lons.append(v.lon)
            
            data.append(
                go.Scattermapbox(lat = lats, lon = lons, 
                             mode = 'markers', 
                             marker = dict(size = self.meta["category_styles"][cat].size,
                                           color = self.meta["category_styles"][cat].color,
                                           opacity = self.meta["category_styles"][cat].opacity
                                          ), 
                             text = texts)
            )
  
        self.meta["data"] = data
        self.meta["layout"] = go.Layout(autosize = True,
                           hovermode = 'closest',
                           mapbox = dict(accesstoken = self.token,
                                         bearing = 0,
                                         center = dict(lat = self.meta["center"].lat,
                                                       lon = self.meta["center"].lon),
                                         pitch = 0,
                                         zoom = 12,
                                         style = self.meta["style"]))

    def iplot (self):
        self.setup_map()
        fig = dict(data = self.meta["data"], layout = self.meta["layout"])
        mapname = "PPPP "  + self.meta["username"] + " - " + self.meta["username"]
        obj = py.iplot(fig, filename = mapname)
        self.meta["url"] = obj.resource
        return obj
    
    def get_url(self):
        if "url" in self.meta:
            return display(HTML('<a href="{0}" target="_blank">{0}</a>'.format(self.meta["url"])))
        else:
            return None